In [25]:
#import packages
import pandas as pd
from nltk.tokenize import TweetTokenizer

## Data Cleaning (preliminary)

In [26]:
tweets = pd.read_pickle("../data/train.pkl")

In [27]:
#isolating relevant columns
tweets.drop(columns=["tweet_id", "text_info_conf", "text_human_conf", "image_id", "image_info", "image_info_conf", "image_human", "image_human_conf", "image_damage", "image_damage_conf", "image_url", "image_path"], inplace=True)

In [28]:
#making text_info binary

#drop rows where text_info == "dont_know_or_cant_judge"
tweets = tweets[tweets.text_info != "dont_know_or_cant_judge"]

#map new values
tweets["info_binary"] = tweets.text_info.map({"informative":1,"not_informative":0})

In [30]:
tweets.info_binary.value_counts(1) #baseline accuracy

1    0.720242
0    0.279758
Name: info_binary, dtype: float64

In [31]:
#make tweets lowercase
lower_tweets = []
for tweet in tweets.tweet_text:
    lower_tweets.append(tweet.lower())

lower_tweets = pd.DataFrame(lower_tweets)

lower_tweets.rename(columns={0:"tweet"}, inplace=True)

tweets.tweet_text = lower_tweets.tweet

#truncate elongations (NO CODE FOR NOW)

#spell out numbers (NO CODE FOR NOW)

#replace all usernames with 'USERID'
tweets.tweet_text = tweets.tweet_text.replace(r'@\S+', 'USERID', regex=True)

#replace all URLs with HTTP
tweets.tweet_text = tweets.tweet_text.replace(r'http\S+', 'HTTP', regex=True).replace(r'www\S+', 'HTTP', regex=True)

#remove "RT" string
tweets.tweet_text = tweets.tweet_text.map(lambda x: x.lstrip('rt'))

## Preprocessing (preliminary)

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
#Name variables 

X = tweets[["tweet_text"]]
y = tweets["info_binary"]

#train test split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42,
                                                    stratify=y)

## Preliminary Model: CountVectorizer and Logistic Regression 

In [39]:
#CountVectorizer 

cvec = CountVectorizer(stop_words="english")
X_train_cvec = pd.DataFrame(cvec.fit_transform(X_train['tweet_text']).todense(), columns = cvec.get_feature_names())
X_test_cvec = pd.DataFrame(cvec.transform(X_test['tweet_text']).todense(), columns = cvec.get_feature_names())
lr = LogisticRegression()
lr_model = lr.fit(X_train_cvec, y_train)
predictions = lr.predict(X_test_cvec)

In [40]:
cross_val_score(lr, X_train_cvec, y_train, cv=5)

array([0.67635468, 0.67175949, 0.66387383, 0.67258383, 0.6622288 ])

In [41]:
lr.score(X_train_cvec, y_train)

0.7978115141955836

In [42]:
lr.score(X_test_cvec, y_test)

0.6658781785925488